## Function Calling Notebook

This notebook walks through the steps to run function calling through OpenAI. Function Calling mimics what Langchain does internally with tools and agents. I think function calling is how chatgpt plugins work through OpenAI.

To get started, please get your API keys for OpenAI and SERPAPI and add those to the environment fine *.env

In [63]:
from dotenv import load_dotenv
load_dotenv()

True

In [45]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from langchain import SerpAPIWrapper, OpenAI, LLMMathChain


GPT_MODEL = "gpt-3.5-turbo-0613"

#### Function for chat completion including functions
The GPT 3.5 Turbo model under 0613 release has support for function calling.
We are going to use OpenAI function calling directly instead of using Langchain
We have defined 2 functions below, one for chat completion and the other to print roles with different colors

In [34]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [35]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    formatted_messages = []
    for message in messages:
        if message["role"] == "system":
            formatted_messages.append(f"system: {message['content']}\n")
        elif message["role"] == "user":
            formatted_messages.append(f"user: {message['content']}\n")
        elif message["role"] == "assistant" and message.get("function_call"):
            formatted_messages.append(f"assistant: {message['function_call']}\n")
        elif message["role"] == "assistant" and not message.get("function_call"):
            formatted_messages.append(f"assistant: {message['content']}\n")
        elif message["role"] == "function":
            formatted_messages.append(f"function ({message['name']}): {message['content']}\n")
    for formatted_message in formatted_messages:
        print(
            colored(
                formatted_message,
                role_to_color[messages[formatted_messages.index(formatted_message)]["role"]],
            )
        )

### Define Function Calls

We are going to have 2 functions
* Search - Google Search for the user query
* Calculator - Function that would do the math calculation

In [36]:
functions = [
    {
        "name": "search",
        "description": "A search engine. Useful for when you need to answer questions about current events. Input should be a search query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query",
                }
            },
            "required": ["query"],
        },
    },
    {
        "name": "calculator",
        "description": "Useful for when you need to answer questions about math.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The math query",
                }
            },
            "required": ["query"]
        },
    },
]

### First Call to OpenAI

We will test the code by passing in the function arguments and making a first call to OpenAI

In [30]:
user_request = """
Find Harry Styles' age. What is their current age, power 2.3?
"""
messages = []
messages.append({"role": "system", "content": "You are a friendly chat assistant."})
messages.append({"role": "user", "content": user_request})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'search',
  'arguments': '{\n  "query": "Harry Styles age"\n}'}}

### Individual function definitions

OpenAI responds with the name of the function and the query to run through it. We need to define the actual functions that will run. 
I am going to leverage Langchain to run these functions easily

In [48]:
def extract_function_information(assistant_message):
    """Function to parse OpenAI assistant response and get information of which function to run."""
    fn_name = assistant_message['function_call']['name']
    search_query = assistant_message['function_call']['arguments']
    search_query = search_query.replace("\n","")
    search_query_dict = json.loads(search_query)
    return fn_name, search_query_dict['query']

def google_search(query):
    """Function to execute Google Search."""
    try:
        search = SerpAPIWrapper()
        results = search.run(query)
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def math_calculator(query):
    """Function to run Math Calculations."""
    try:
        llm = OpenAI(temperature=0)
        calculator = LLMMathChain.from_llm(llm=llm)
        results = calculator.run(query)
    except Exception as e:
        results = f"query failed with error: {e}"
    return results


def function_executor(assistant_message):
    """Tie above functions together so either can be executed"""
    name, query = extract_function_information(assistant_message)
    if name == 'search':
        results = google_search(query)
    elif name == 'calculator':
        results = math_calculator(query)
    else:
        results = f"Error: function {name} does not exist"
    return results

Now run this all together with a while loop that keeps sending messages to OpenAI till the user has their answer

In [52]:
user_request = """
Find Harry Styles' age. What is their current age power 2.3?
"""
messages = []
messages.append({"role": "system", "content": "You are a friendly chat assistant."})
messages.append({"role": "user", "content": user_request})
keep_running = True
while keep_running:
    chat_response = chat_completion_request(
        messages, functions=functions
    )
    assistant_message = chat_response.json()["choices"][0]["message"]
    messages.append(assistant_message)
    if assistant_message.get("function_call"):
        results = function_executor(assistant_message)
        messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})
        keep_running = True
    else:
        keep_running = False
pretty_print_conversation(messages)

system: You are a friendly chat assistant.

user: 
Find Harry Styles' age. What is their current age power 2.3?


assistant: {'name': 'search', 'arguments': '{\n  "query": "Harry Styles age"\n}'}

function (search): 29 years

assistant: {'name': 'calculator', 'arguments': '{\n  "query": "29 power 2.3"\n}'}

function (calculator): Answer: 2309.486325717843

assistant: Harry Styles is currently 29 years old. Their current age to the power of 2.3 is approximately 2309.49.



Try again with a more complex request

In [55]:
def openai_function_example(user_query):
    messages = []
    messages.append({"role": "system", "content": "You are a friendly chat assistant."})
    messages.append({"role": "user", "content": user_request})
    keep_running = True
    while keep_running:
        chat_response = chat_completion_request(
            messages, functions=functions
        )
        assistant_message = chat_response.json()["choices"][0]["message"]
        messages.append(assistant_message)
        if assistant_message.get("function_call"):
            results = function_executor(assistant_message)
            messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})
            keep_running = True
        else:
            keep_running = False
    pretty_print_conversation(messages)
    return messages

In [57]:
user_request = """
How old is Donald trump in 2023 ? How old is Biden in 2023 and who is younger by how much?
"""
messages = openai_function_example(user_request)

system: You are a friendly chat assistant.

user: 
How old is Donald trump in 2023 ? How old is Biden in 2023 and who is younger by how much?


assistant: To determine the age of Donald Trump and Joe Biden in 2023, we need to know their birth dates. However, as of now, it is 2021 and their birth dates are well-known. 

Donald Trump was born on June 14, 1946, and Joe Biden was born on November 20, 1942. 

In 2023: 

- Donald Trump will be 77 years old (assuming his birthday has already passed). 
- Joe Biden will be 81 years old (assuming his birthday has already passed). 

Therefore, in 2023, Joe Biden will be older than Donald Trump by 4 years.



In [62]:
user_request = """
What is current US unemployment rate? If it decreases by 25%, what will it be then?
"""
messages = openai_function_example(user_request)

system: You are a friendly chat assistant.

user: 
What is current US unemployment rate? If it decreases by 25%, what will it be then?


assistant: {'name': 'search', 'arguments': '{\n  "query": "current US unemployment rate"\n}'}

function (search): The unemployment rate increased by 0.3 percentage point to 3.7 percent in May, and the number of unemployed persons rose by 440,000 to 6.1 ...

assistant: The current US unemployment rate is 3.7%. If it decreases by 25%, the new unemployment rate will be approximately 2.775%.

